<a href="https://colab.research.google.com/github/poojamahajan0712/GenAI/blob/main/LLM/3_opensource_LLM_Mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## prerequisites -- hf account, access token and request for model access based on model.
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 10.1 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
# Double quantization is an additional compression technique where the quantized weights are themselves further quantized, reducing storage and computational overhead.

In [4]:
from huggingface_hub import login

login("<key>")

In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2",
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [8]:
pipe = pipeline("text-generation",
    model=model,
    tokenizer = tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

## model quantized to 4 bit and calculations will happen in bf16 when dequantized

Device set to use cuda:0


## general QA

In [9]:
prompt = "Can you explain the difference between equity and equality"
# temperature increases variety, while top-p and top-k reduce variety and focus samples on the model’s top predictions.
# Higher temperature will make outputs more random and diverse.
# Lower top-p values reduce diversity and focus on more probable tokens.
# Lower top-k also concentrates sampling on the highest probability tokens for each step.

sequences = pipe(prompt,
    do_sample=True,
    max_new_tokens=500,
    temperature=0.7,
    top_k=50, # Uses top-k sampling, meaning only the top 50 most likely words are considered at each step
    top_p=0.95,
    num_return_sequences=1,)
print(sequences[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Can you explain the difference between equity and equality?

Equity and equality are related concepts, but they are not the same thing. Equality refers to the idea that everyone should be treated the same, regardless of their characteristics, such as race, gender, or socioeconomic status. Equity, on the other hand, recognizes that people have different needs and require different resources to achieve the same outcomes. Therefore, equity involves providing resources and opportunities in a way that addresses the unique needs of individuals or groups to ensure fairness.

For example, in an educational context, equality might mean that all students have access to the same textbooks and class materials. Equity, however, might mean that students who are English language learners receive additional resources, such as bilingual teachers or translators, to ensure they have an equal opportunity to learn.

In summary, equality focuses on sameness and uniformity, while equity recognizes and addres

##math knowledge

** wrong

In [11]:
prompt = "Solve this quadratic equation for x in x*x-9 =7"
sequences = pipe(prompt,
    do_sample=True,
    max_new_tokens=500,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,)
print(sequences[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Solve this quadratic equation for x in x*x-9 =7:

$x^2-9=7$

First, we need to get the equation in standard form, which is $ax^2+bx+c=0$.

To do that, we need to move the constant term to the right side of the equation:

$x^2-9-7=0$

Now, we have $x^2-9+7=0$.

Adding 9 to both sides:

$x^2-9+9=9$

Which simplifies to:

$x^2=10$

Now, we need to find the solutions to $x^2=10$.

Since this is a perfect square trinomial, we can take the square root of both sides:

$\pm\sqrt{x^2}=\pm\sqrt{10}$

So, $x=\pm\sqrt{10}$.

Therefore, the solutions to the quadratic equation are:

$x=\pm\sqrt{10}$

Or, equivalently:

$x=\pm 3.16227766\text{ (rounded to 7 decimal places)}$

Note: In practice, we often do not calculate the square root of 10 to 7 decimal places, but it's important to understand the theoretical solution.


## planning
** pretty decent

In [12]:
prompt = """I have three books No bullshit guide to linear algebra, designing machine learning system and AI engineering. can you create
reading plan considering 30 min per day and help me decide which book i should start with? """
sequences = pipe(prompt,
    do_sample=True,
    max_new_tokens=2000,
    temperature=0.5,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,)
print(sequences[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


I have three books No bullshit guide to linear algebra, designing machine learning system and AI engineering. can you create 
reading plan considering 30 min per day and help me decide which book i should start with? 

To create a reading plan for your given books and time constraint, I'd suggest the following approach:

1. Understand the core concepts: Since all three books have some overlap in terms of linear algebra and machine learning, it would be beneficial to start with a strong foundation in these areas. I recommend starting with "No Bullshit Guide to Linear Algebra" as it's a concise and straightforward introduction to the subject. Plan to spend 1-2 weeks (around 15 days) on this book, aiming for 30 minutes per day.

2. Machine Learning System: After building a strong foundation in linear algebra, you can move on to "Designing Machine Learning Systems" by Andrew Ng. This book covers the practical aspects of designing and implementing machine learning systems. Spend around 3 we

In [13]:
prompt = """I want to learn a new language can you help me create a plan and ideas,
 i am familair with english and hindi. And the new
language is Kannada"""
sequences = pipe(prompt,
    do_sample=True,
    max_new_tokens=2000,
    temperature=0.5,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,)
print(sequences[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


I want to learn a new language can you help me create a plan and ideas, i am familair with english and hindi. And the new
language is Kannada.

Certainly! Learning a new language can be an exciting and rewarding experience. Here's a plan and some ideas to help you get started with learning Kannada:

1. Set clear goals: Decide why you want to learn Kannada and set specific, measurable, achievable, relevant, and time-bound (SMART) goals. For example, you might want to be able to hold a basic conversation with a Kannada speaker within six months.
2. Find resources: Look for resources to help you learn Kannada. This could include language learning apps, textbooks, audio and video materials, and language exchange partners.
3. Practice regularly: Consistency is key when learning a new language. Set aside time each day to practice, even if it's just for 15 minutes.
4. Focus on the basics: Start by learning the alphabet, numbers, and basic grammar rules. This will give you a solid foundation f

## code debug

In [14]:
prompt = """can debug what is wrong with this factorial code -
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n - 2)
"""
sequences = pipe(prompt,
    do_sample=True,
    max_new_tokens=2000,
    temperature=0.5,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,)
print(sequences[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


can debug what is wrong with this factorial code - 
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n - 2) 

This code is supposed to calculate the factorial of a given number n. However, it has an error in it. The base case is incorrect. It should be n == 1 instead of n == 0.

Here's the corrected code:

def factorial(n):
    if n == 1:
        return 1
    else:
        return n * factorial(n - 1) 

This corrected code will now correctly calculate the factorial of any given number.


In [15]:
prompt = """ can you generate a python script for merging two jpg together and final output as jpg
"""
sequences = pipe(prompt,
    do_sample=True,
    max_new_tokens=2000,
    temperature=0.5,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,)
print(sequences[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 can you generate a python script for merging two jpg together and final output as jpg

Yes, here's a simple Python script using the Pillow library to merge two JPG images into a single JPG image:

```python
from PIL import Image

# Open the two images to be merged
image1 = Image.open('image1.jpg')
image2 = Image.open('image2.jpg')

# Resize the second image if necessary to match the size of the first image
width, height = image1.size
image2 = image2.resize((width, height), Image.ANTIALIAS)

# Merge the two images
merged_image = Image.merge('RGB', (image1, image2))

# Save the merged image as a JPG file
merged_image.save('merged_image.jpg')
```

Replace `'image1.jpg'` and `'image2.jpg'` with the file paths of the two images you want to merge. The merged image will be saved as `'merged_image.jpg'` in the same directory as the Python script.

Make sure you have the Pillow library installed before running the script. You can install it using pip:

```
pip install Pillow
```


In [17]:

prompt = """ debug this 'PIL.Image' has no attribute 'ANTIALIAS'
"""
sequences = pipe(prompt,
    do_sample=True,
    max_new_tokens=2000,
    temperature=0.5,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,)
print(sequences[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 debug this 'PIL.Image' has no attribute 'ANTIALIAS'

Here's the corrected code:

```python
from PIL import Image, ImageFilter

def resize_image(image, size):
    return image.resize(size, Image.ANTIALIAS)

def resize_thumbnail(image, size):
    return image.thumbnail(size, Image.ANTIALIAS)

def apply_filter(image, filter_name):
    if filter_name == "BLUR":
        return image.filter(ImageFilter.BLUR)
    elif filter_name == "SHARPEN":
        return image.filter(ImageFilter.SHARPEN)
    else:
        raise ValueError("Invalid filter name")

image = Image.open("image.jpg")

resized_image = resize_image(image, (200, 200))
thumbnail = resize_thumbnail(image, (100, 100))
filtered_image = apply_filter(image, "BLUR")
```

The issue was that `Image` class from PIL module does not have an attribute named `ANTIALIAS`. Instead, it should be used as a method argument when calling `resize()` or `thumbnail()` functions. Therefore, I corrected the `resize_image()` and `resize_thumbnail()` functio

In [16]:
from PIL import Image

# Open the two images to be merged
image1 = Image.open('image1.jpg')
image2 = Image.open('image2.jpg')

# Resize the second image if necessary to match the size of the first image
width, height = image1.size
image2 = image2.resize((width, height), Image.ANTIALIAS)

# Merge the two images
merged_image = Image.merge('RGB', (image1, image2))

# Save the merged image as a JPG file
merged_image.save('merged_image.jpg')

AttributeError: module 'PIL.Image' has no attribute 'ANTIALIAS'